**converting_WRF_ouput_netcdf_into_json**

- tried with grib2json for converting netcdf into json [1](https://github.com/cambecc/grib2json)
- for this netcdf has to converted into grib2
- for converting into grib2, python based iris is useful [2](http://scitools.org.uk/iris/docs/latest/installing.html), but only work with cf compliant netcdf
- WRF output in netcdf is not a cf compliant
- So has to use a tool which convert WRF netcdf into CF compliant
- Wrfncxnj.py tool [3](http://www.meteo.unican.es/wiki/cordexwrf/SoftwareTools/WrfncXnj) exactly do this with more functions such as extraction of variables etc
- Wrfncxnj.py is python based tool, using that extracted wind at 2m from WRF output 
```
python wrfncxnj.py -v SPDUV10 -o wind.nc wrfout_d04_2014-03-20_13:00:00
```
- to extract various variables such as temperature, relative humidity, sea surface pressure, dew point temperature, precepitation etc
```
python wrfncxnj.py --split-variables --output-pattern=Zwrfout_[varcf].nc -v "T2,Q2,TDPS,MSLP,RAIN,WIND" wrfout_d04_2014-03-20_13:00:00
``` 
- to convert this netcdf into a json used ncdump-json [4](https://github.com/jllodra/ncdump-json)
- compiled ncdump-json as told in README and it works perfectly.
- command 
```
ncdump-json wind.nc -h >>wind.json
``` 
- extracts header file of nc and write into a json file, the command is 
```
ncdump-json wind.nc -j >>wind.json
``` 
- extract data of nc and write into previous file down of header details.

**Now the work flow using python script**
- the command gives error for multiple wrfout files execution
```
python wrfncxnj.py --split-variables --output-pattern=Zwrfout_[varcf].nc -v "T2,Q2,TDPS,MSLP,RAIN,WIND" wrfout_d04_2014-03-20_13:00:00
``` 
- So using python os module to get the wrf out file in a folder and execute this command one by one for all the files in that particular folder.
- the command to extract file names in folder is 
```
import os
data_files = [(x[0], x[2]) for x in os.walk('/home/swl-sacon-dst/Documents/GISE_2013/LAB/GRIBJSON_inst/wrfncxnj-0.1_r2120/1/')]
```
- this gives a list of list with parent directory and list of files in that directory, removing x[0] prints out list of list with only file name
- so iterating list of list using for loop gives, unredered list, here insteaad of print executing sub process with the for loop n can execute the wrfncxnj.py for each file. 
```
for list in data_files:
        for n in list:
            print n
```
- To call external python script in another python script use
```
for list in data_files:
...         for n in list:
...             subprocess.call('python wrfncxnj.py --split-variables --output-pattern='+n+'_[varcf].nc -v "T2,Q2,TDPS,MSLP,RAIN,WIND" '+n, shell=True)
```
- for testing used
```
print 'python wrfncxnj.py --split-variables --output-pattern='+n+'_[varcf].nc -v "T2,Q2,TDPS,MSLP,RAIN,WIND" '+n
```
- To change directory in python program to get inside compiled directory of ncdump-json, used
```
 os.chdir('/home/swl-sacon-dst/Documents/GISE_2013/LAB/ncdump-json/ncdump-json-master/')
```
- to run the ncdump-json and save output json in a variable, used
```
proc = subprocess.Popen('ncdump-json wind.nc -j', shell=True, stdout=subprocess.PIPE)
output = proc.stdout.read()
print output
```
- to find nearest number in a list,
```
myList = [4,1,88,44,3]
myNumber = 5
min(myList, key=lambda x:abs(x-myNumber))
```
- To query a json file in python
```
import json
 with open('/home/swl-sacon-dst/Documents/GISE_2013/LAB/cbeair-edit/earth/public/data/weather/current/current-wind-surface-level-gfs-1.0.json') as f:
		 data = json.load(f)
print(data[0]['header'])
```